<a href="https://colab.research.google.com/github/P-24654/Disease-Prediction-using-Catboost/blob/main/catboost_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install catboost
%pip install scispacy
%pip install spacy
%pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_md-0.5.1.tar.gz
%pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 13.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 18.3 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13571729 sha256=74094134a17d335e2f6008cd77ffec5279c25f4c86e1d1e16b537b82d62cfa72
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/Training.csv")

In [ ]:
df.head()

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis,Unnamed: 133
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Fungal infection,NaN


In [ ]:
df.shape

(4920, 134)

In [ ]:
df.drop('Unnamed: 133', axis=1, inplace=True)
df.columns

Index(['itching', 'skin_rash', 'nodal_skin_eruptions', 'continuous_sneezing',
       'shivering', 'chills', 'joint_pain', 'stomach_pain', 'acidity',
       'ulcers_on_tongue',
       ...
       'blackheads', 'scurring', 'skin_peeling', 'silver_like_dusting',
       'small_dents_in_nails', 'inflammatory_nails', 'blister',
       'red_sore_around_nose', 'yellow_crust_ooze', 'prognosis'],
      dtype='object', length=133)

In [ ]:
df['prognosis'].value_counts()

Fungal infection                           120
Hepatitis C                                120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Dimorphic hemmorhoids(piles)               120
Heart attack                               120
Varicose veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis B                                120
Allergy      

In [ ]:
x = df.drop('prognosis', axis = 1)
y = df['prognosis']

df_test = pd.read_csv("/content/Testing.csv")
x_test = df_test.drop('prognosis', axis = 1)
y_test = df_test['prognosis']

In [ ]:
from catboost import CatBoostClassifier
catboost_classifier = CatBoostClassifier(iterations=18, task_type="GPU", devices='0' ,learning_rate=0.1, depth=6, loss_function='MultiClass', eval_metric='Accuracy', auto_class_weights='Balanced')


In [ ]:
catboost_classifier.fit(x, y,eval_set=(x_test, y_test), use_best_model=True, verbose=True)

CatBoostError: ignored

In [ ]:
pred = catboost_classifier.predict(x_test)
acc = catboost_classifier.score(x_test, y_test)

print("Acurray on test set: {:.2f}%".format(acc*100))

Acurray on test set: 97.62%


In [ ]:
import en_core_sci_md
nlp_md = en_core_sci_md.load()

In [ ]:
def get_entities(text):
    entity = nlp_md(text)
    return [(ent.text, ent.label_) for ent in entity.ents]

In [ ]:
input_text = input()
entities = get_entities(input_text)
entities

chills, shivering, watering from eyes


[('chills', 'ENTITY'),
 ('shivering', 'ENTITY'),
 ('watering', 'ENTITY'),
 ('eyes', 'ENTITY')]

In [ ]:
# get the similarity between 2 words

# from scipy import spatial

# def get_similarity(word1, word2):
#     return 1 - spatial.distance.cosine(nlp_md(word1).vector, nlp_md(word2).vector)

from difflib import SequenceMatcher

def get_similarity(w1, w2):
  return SequenceMatcher(None, w1, w2).ratio()

In [ ]:
features = x.columns
test_input = pd.DataFrame(columns=features)
test_input.loc[len(test_input)] = np.nan

for (ent, _) in entities:
    similarities = [get_similarity(ent, feature) for feature in features]
    max_feature = features[np.argmax(similarities)]
    test_input[max_feature] = 1

test_input = test_input.fillna(0)

In [ ]:

import numpy as np

from numpy import dot
from numpy.linalg import norm
from transformers import AutoTokenizer, AutoModel


def get_bert_based_similarity(sentence_pairs):
    """
    computes the embeddings of each sentence and its similarity with its corresponding pair
    Args:
        sentence_pairs(dict): dictionary of lists with the similarity type as key and a list of two sentences as value
        model: the language model
        tokenizer: the tokenizer to consider for the computation
    
    Returns:
        similarities(dict): dictionary with similarity type as key and the similarity measure as value
    """
    model = AutoModel.from_pretrained('dmis-lab/biobert-v1.1')
    tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-v1.1')

    similarities = dict()
    for sim_type, sent_pair in sentence_pairs.items():
        inputs_1 = tokenizer(sent_pair[0], return_tensors='pt')
        inputs_2 = tokenizer(sent_pair[1], return_tensors='pt')
        sent_1_embed = np.mean(model(**inputs_1).last_hidden_state[0].detach().numpy(), axis=0)
        sent_2_embed = np.mean(model(**inputs_2).last_hidden_state[0].detach().numpy(), axis=0)
        similarities[sim_type] = dot(sent_1_embed, sent_2_embed)/(norm(sent_1_embed)* norm(sent_2_embed))
    return similarities

In [ ]:
features = x.columns
test_input = pd.DataFrame(columns=features)
test_input.loc[len(test_input)] = np.nan

for (ent, _) in entities:
    similarities = [get_bert_based_similarity({"similar": [ent, feature]})['similar'] for feature in features]
    max_feature = features[np.argmax(similarities)]
    test_input[max_feature] = 1

test_input = test_input.fillna(0)

In [ ]:
from google.colab.data_table import DataTable
DataTable.max_columns = 132

In [ ]:
test_input

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pred = catboost_classifier.predict(test_input)

In [ ]:
pred

array([['Heart attack']], dtype=object)